In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import random

e:\deep-learning-on-practice-NLP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_cv = pd.read_csv("../data/datasets_with_text/cv.csv")
df_vac = pd.read_csv("../data/datasets_with_text/vac.csv")

In [3]:
df_cv.head()

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,cv_text
0,0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,"Пол, возраст:\nМужчина , 39 лет , родился 27 ..."
1,1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,"Пол, возраст:\nМужчина , 60 лет , родился 20 ..."
2,2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,"Пол, возраст:\nЖенщина , 36 лет , родилась 12..."
3,3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,"Пол, возраст:\nМужчина , 38 лет , родился 25 ..."
4,4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано,"Пол, возраст:\nЖенщина , 26 лет , родилась 3 ..."


In [4]:
df_vac.head()

,Unnamed: 0,id,Сфера деятельности,spec_id,Должность,Требования,Обязанности,Навыки,Описание вакансии,vac_text
0,0,10176972,"Программирование, Разработка",221,Ведущий инженер-программист (Delphi),"Знание основ ООП, Delphi XE и выше, основных ш...",Не указано,Не указано,"Требования: Знание основ ООП, Delphi XE и выш...","Сфера деятельности:\nПрограммирование, Разрабо..."
1,1,15518045,Игровое ПО,475,Программист графики,Желание изучать и внедрять новые технологии. С...,"Работа с текущим поколением консолей Xbox 1, P...","OpenGL,C++,Графика,Rendering,DirectX,Vulkan",Привет! Наша игровая студия Saber Interactive ...,Сфера деятельности:\nИгровое ПО\nДолжность:\nП...
2,2,16449260,Продажи,225,Специалист по недвижимости,Грамотная устная и письменная речь. Дисциплина...,Все операции (кроме аренды) на рынке жилой нед...,Не указано,За 7 лет работы Мы стали лидерами на рынке нед...,Сфера деятельности:\nПродажи\nДолжность:\nСпец...
3,3,16897422,Продажи,225,Специалист по продажам услуг,Не указано,Работаешь на выделенной территории. Распростра...,Не указано,МТС – одна из крупнейших российских экосистем....,Сфера деятельности:\nПродажи\nДолжность:\nСпец...
4,4,17913733,Продажи,225,Специалист по продажам услуг,Не указано,Работаешь на выделенной территории. Распростра...,Не указано,МТС – одна из крупнейших российских экосистем....,Сфера деятельности:\nПродажи\nДолжность:\nСпец...


In [6]:
qa_pipeline = pipeline(
    "question-answering",
    model="Den4ikAI/rubert-large-squad",
    tokenizer="Den4ikAI/rubert-large-squad"
)

qa_pipeline2 = pipeline(
    "question-answering",
    model="AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru"
)

qa_pipeline3 = pipeline(
    "question-answering",
    model="timpal0l/mdeberta-v3-base-squad2"
)


config.json: 100%|██████████| 879/879 [00:00<00:00, 1.55MB/s]
e:\deep-learning-on-practice-NLP\venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yxtag\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
model.safetensors: 100%|██████████| 1.11G/1.11G [01:38<00:00, 11.3MB/s]
to

Возьмем рандомное резюме и попытаемся вытащить из него любую полезную информацию

In [32]:
cv_num = random.randint(0, len(df_cv) - 1)
vac_num = random.randint(0, len(df_vac) - 1)

In [34]:
cv = df_cv.iloc[cv_num]["cv_text"]
vac = df_vac.iloc[vac_num]["vac_text"]

In [35]:
print(cv)

Пол, возраст:
Мужчина ,  30 лет , родился 21 ноября 1988
ЗП:
1000 USD
Ищет работу на должность::
Менеджер проектов
Город, переезд, командировки:
Киев ,  м. Олимпийская , не готов к переезду , готов к командировкам
Занятость:
частичная занятость, проектная работа, полная занятость
График:
гибкий график, полный день, удаленная работа
Опыт работы:
Опыт работы 8 лет 1 месяц  Менеджер проектов 1 000 USD Информационные технологии, интернет, телеком Управление проектами Занятость: частичная занятость, проектная работа, полная занятость График работы: гибкий график, полный день, удаленная работа Опыт работы 8 лет 1 месяц Июнь 2016 — по настоящее время 2 года 11 месяцев ESM.one Украина , esm.one Информационные технологии, системная интеграция, интернет ... Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее) Исполнительный продюсер Ведение проектов компании от этапа утверждения до реализации. Январь 2016

Пробуем достать должность

In [36]:
qa_pipeline({
    'context': cv,
    'question': "На какую должность ищет работу?"
})

{'score': 0.9508810639381409,
 'start': 97,
 'end': 120,
 'answer': 'Менеджер проектов\nГород'}

In [37]:
qa_pipeline2({
    'context': cv,
    'question': "На какую должность ищет работу?"
})

{'score': 0.8721234798431396,
 'start': 96,
 'end': 114,
 'answer': '\nМенеджер проектов'}

In [38]:
qa_pipeline3({
    'context': cv,
    'question': "На какую должность ищет работу?"
})

{'score': 0.9036442637443542,
 'start': 96,
 'end': 114,
 'answer': '\nМенеджер проектов'}

Пробуем достать место работы

In [39]:
qa_pipeline({
    'context': cv,
    'question': "В каком городе ищет работу?"
})

{'score': 0.6461823582649231, 'start': 145, 'end': 149, 'answer': 'Киев'}

In [40]:
qa_pipeline2({
    'context': cv,
    'question': "В каком городе ищет работу?"
})

{'score': 0.5436960458755493,
 'start': 3263,
 'end': 3279,
 'answer': ' Санкт-Петербург'}

In [41]:

qa_pipeline3({
    'context': cv,
    'question': "В каком городе ищет работу?"
})


{'score': 0.967211902141571, 'start': 144, 'end': 149, 'answer': '\nКиев'}

Пробуем достать навыки

In [42]:
qa_pipeline({
    'context': cv,
    'question': "Какими навыками/скиллами обладает?"
})

{'score': 0.4756474196910858,
 'start': 2067,
 'end': 2082,
 'answer': 'Ключевые навыки'}

In [43]:
qa_pipeline2({
    'context': cv,
    'question': "Какими навыками/скиллами обладает?"
})

{'score': 0.23754167556762695,
 'start': 191,
 'end': 213,
 'answer': ' готов к командировкам'}

In [44]:
qa_pipeline3({
    'context': cv,
    'question': "Какими навыками/скиллами обладает?"
})

{'score': 0.20796816051006317,
 'start': 401,
 'end': 467,
 'answer': ' Информационные технологии, интернет, телеком Управление проектами'}

Достанем желаемую зарплату

In [45]:
qa_pipeline({
    'context': cv,
    'question': "Какая желаемая зарплата?"
})

{'score': 0.990283191204071, 'start': 61, 'end': 69, 'answer': '1000 USD'}

In [46]:
qa_pipeline2({
    'context': cv,
    'question': "Какая желаемая зарплата?"
})

{'score': 0.8587424755096436, 'start': 60, 'end': 69, 'answer': '\n1000 USD'}

In [47]:
qa_pipeline3({
    'context': cv,
    'question': "Какая желаемая зарплата?"
})

{'score': 0.44397616386413574, 'start': 60, 'end': 69, 'answer': '\n1000 USD'}

Пробуем достать желаемый график работы

In [51]:
qa_pipeline({
    'context': cv,
    'question': "Какой графики работы устраивают автора?"
})

{'score': 0.4150893986225128,
 'start': 289,
 'end': 333,
 'answer': 'гибкий график, полный день, удаленная работа'}

In [52]:
qa_pipeline2({
    'context': cv,
    'question': "Какой графики работы устраивают автора?"
})

{'score': 0.2155931144952774, 'start': 288, 'end': 295, 'answer': '\nгибкий'}

In [53]:
qa_pipeline3({
    'context': cv,
    'question': "Какой графики работы устраивают автора?"
})

{'score': 0.04510054737329483,
 'start': 288,
 'end': 333,
 'answer': '\nгибкий график, полный день, удаленная работа'}